In [1]:
import pandas as pd
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
import pickle as pkl
from keras.optimizers import SGD
import keras.backend as K
from densenet161 import DenseNet 
import numpy as np 
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.models import load_model
import cv2
import numpy as np
from keras.optimizers import SGD
import keras.backend as K
from keras.models import Model
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization 
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import pickle as pkl
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.callbacks import TensorBoard
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)  
KTF.set_session(sess)

Using TensorFlow backend.


In [2]:
nb_class=29

In [3]:
def process_pic(image):
    image=cv2.resize(image,(224,224))
#     image = img_to_array(image)
#     image[:,:,0] = (image[:,:,0] - 103.94) * 0.017
#     image[:,:,1] = (image[:,:,1] - 116.78) * 0.017
#     image[:,:,2] = (image[:,:,2] - 123.68) * 0.017
    return image

In [4]:
def get_data(image_path_origin="data/train",image_path_gan="data/gen_class/",label_path="data/Train_label.csv"):
    train_label = pd.read_csv("data/Train_label.csv")
    arr=train_label.values
    image_dict={}
    for i in range(arr.shape[0]):
        tmp_arr=list(map(int,arr[i][1].split(';')))
        for j in range(len(tmp_arr)):
            tmp_arr[j]=tmp_arr[j]-1
        image_dict[arr[i][0]]=tmp_arr
#         print(image_dict[arr[i][0]])
        # image_dict[arr[i][0]]=int(arr[i][1].split(';')[0])-1
    data=[]
    label=[]
    image_paths_origin = sorted(list(paths.list_images(image_path_origin)))
    for im_path in image_paths_origin:
        image=cv2.imread(im_path)
#         if image.shape[0]*image.shape[1]<448*448:
        if True:
            image=cv2.resize(image,(224,224))
#             image = img_to_array(image)
#             image[:,:,0] = (image[:,:,0] - 103.94) * 0.017
#             image[:,:,1] = (image[:,:,1] - 116.78) * 0.017
#             image[:,:,2] = (image[:,:,2] - 123.68) * 0.017
            tmp_image_name=im_path.split('/')[2]
            data.append(image)
            label.append(image_dict[tmp_image_name])
        else:
            half_h=int(image.shape[0]/2)
            half_w=int(image.shape[1]/2)
            image_1=image[0:half_h,0:half_w,:]
            image_2=image[half_h:-1,0:half_w,:]
            image_3=image[0:half_h,half_w:-1,:]
            image_4=image[half_h:-1,half_w:-1,:]
            image_1=process_pic(image_1)
            image_2=process_pic(image_2)
            image_3=process_pic(image_3)
            image_4=process_pic(image_4)
            tmp_image_name=im_path.split('/')[2]
            data.append(image_1)
            data.append(image_2)
            data.append(image_3)
            data.append(image_4)
            label.append(image_dict[tmp_image_name])
            label.append(image_dict[tmp_image_name])
            label.append(image_dict[tmp_image_name])
            label.append(image_dict[tmp_image_name])
    #step two add gan pic
    output_data = open('data.pkl', 'wb')
    output_label= open('label.pkl','wb')
    pkl.dump(data,output_data)
    pkl.dump(label,output_label)

In [5]:
get_data()

In [6]:
def load_data(data_path='data.pkl',label_path='label.pkl'):
    pkl_data_file = open('data.pkl', 'rb')
    pkl_label_file=open('label.pkl','rb')
    data = pkl.load(pkl_data_file)
    label=pkl.load(pkl_label_file)
    return data,label

In [7]:
def predict_data(p_path="data/test",model_path="imagenet_models/new_dense_weight.h5"):
	image_paths = sorted(list(paths.list_images(p_path)))
	data=[]
	image_name=[]
	for im_path in image_paths:
		image=cv2.imread(im_path)
		image=cv2.resize(image,(224,224))
		image = img_to_array(image)
		image[:,:,0] = (image[:,:,0] - 103.94) * 0.017
		image[:,:,1] = (image[:,:,1] - 116.78) * 0.017
		image[:,:,2] = (image[:,:,2] - 123.68) * 0.017
		data.append(image)
		image_name.append(im_path.split('/')[2])
	data = np.array(data, dtype="float32")#transfer to numpy
	model = DenseNet(reduction=0.5, classes=nb_class, weights_path=model_path)
	sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
	model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
	out= model.predict(data)
	label=np.argmax(out[0])
	print(type(label))
	print(label)
	print(label.shape)
	predict_label = pd.read_csv("data/submit_example.csv")
	for i in range(predict_label.values.shape[0]):
		find_name=predict_label.values[i][0]
		for j in range(data.shape[0]):
			if find_name==image_name[j]:
				predict_label.iat[i,1]=np.argmax(out[j])+1
	predict_label.to_csv("data/predict.csv",index=False)

In [8]:

def load_new_model(weight_path='imagenet_models/densenet161_weights_tf.h5'):
	base_model = DenseNet(reduction=0.5, classes=1000, weights_path=weight_path)
	# model = Model(input=base_model.input, output=base_model.get_layer('conv5_blk_scale').output)
	x=base_model.get_layer('conv5_blk_scale').output
	x = Activation('relu', name='relu_last')(x)
	x=GlobalAveragePooling2D(name='pool_last')(x)
	x= Dense(nb_class, name='fc_last')(x)
	x=Activation('softmax', name='prob_last')(x)
	model=Model(input=base_model.input,output=x,name='new_densenet')
	for layer in model.layers[:-5]:
	    layer.trainable = False
	return model



In [9]:
def train_data(data,label,model):
    (trainX, testX, trainY, testY) = train_test_split(data,
        label, test_size=0.1, random_state=42)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#     datagen = ImageDataGenerator(
#         shear_range=0.2, # 随机错切换角度
#         zoom_range=0.2, # 随机缩放范围
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
#         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=True,  # randomly flip images
#         vertical_flip=False)  # randomly flip images
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    print(trainX.shape[0])
    batchs=30
#     model.fit_generator(datagen.flow(trainX, trainY, batch_size=batchs),steps_per_epoch=trainX.shape[0], 
#                         epochs=3, verbose=1)
    model.fit(trainX,trainY,batch_size=50,epochs=15,verbose=1,validation_data=(testX,testY),callbacks=[TensorBoard(log_dir='/logss')])
    model.save_weights('imagenet_models/new_dense_weight.h5')
    model.save('imagenet_models/new_densenet.h5')
    score = model.evaluate(testX,testY,verbose=0)
    print ('accuracy:{}'.format(score[1]))

In [10]:
def predict_data(p_path="data/test",model_path="imagenet_models/new_dense_weight.h5"):
    image_paths = sorted(list(paths.list_images(p_path)))
    data=[]
    image_name=[]
    for im_path in image_paths:
        image=cv2.imread(im_path)
        image=cv2.resize(image,(224,224))
        image = img_to_array(image)
        image[:,:,0] = (image[:,:,0] - 103.94) * 0.017
        image[:,:,1] = (image[:,:,1] - 116.78) * 0.017
        image[:,:,2] = (image[:,:,2] - 123.68) * 0.017
        data.append(image)
        image_name.append(im_path.split('/')[2])
    data = np.array(data, dtype="float32")#transfer to numpy
    model = DenseNet(reduction=0.5, classes=nb_class, weights_path=model_path)
    #origin ls is 1e-2
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    out= model.predict(data)
    return out
# 	label=np.argmax(out[0])
# 	print(type(label))
# 	print(label)
# 	print(label.shape)
# 	predict_label = pd.read_csv("data/submit_example.csv")
# 	for i in range(predict_label.values.shape[0]):
# 		find_name=predict_label.values[i][0]
# 		for j in range(data.shape[0]):
# 			if find_name==image_name[j]:
# 				predict_label.iat[i,1]=np.argmax(out[j])+1
# 	predict_label.to_csv("data/predict.csv",index=False)

In [11]:
data,label=load_data()

In [12]:
data_out=np.zeros((len(data),224,224,3),dtype="float32")

In [13]:
test=data[0]
test[:,:,0]

array([[158, 158, 158, ..., 152, 152, 152],
       [158, 158, 158, ..., 152, 152, 152],
       [156, 156, 156, ..., 152, 152, 152],
       ..., 
       [ 26,  26,  24, ...,  46,  46,  46],
       [ 26,  26,  25, ...,  45,  47,  43],
       [ 26,  26,  22, ...,  44,  45,  42]], dtype=uint8)

In [14]:
for i in range(len(data)):
    tmp_image=data[i]
    data_out[i,:,:,0] = (tmp_image[:,:,0] - 103.94) * 0.017
    data_out[i,:,:,1] = (tmp_image[:,:,1] - 116.78) * 0.017
    data_out[i,:,:,2] = (tmp_image[:,:,2] - 123.68) * 0.017

In [15]:
# load data then train data
# data,label=load_data()
print(len(label))
label_out=np.zeros((len(label),nb_class))
for i in range(len(label)):
    t=0
    for j in range(len(label[i])):
        tmp=label[i][j]
        tmp=np.array(tmp)
        tmp=np_utils.to_categorical(tmp,nb_class)
        t=t+tmp
        label_out[i]=t
# data_out = np.array(data, dtype="float32")
model=load_new_model()
train_data(data_out,label_out,model)

10665


/root/workspace/densenet161.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), name="conv1", strides=(2, 2), use_bias=False)`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv2_1_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv2_1_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:165: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'

9598
Train on 9598 samples, validate on 1067 samples
Epoch 1/15
9598/9598 [==============================] - 153s - loss: 2.5491 - acc: 0.2822 - val_loss: 2.2752 - val_acc: 0.3468
Epoch 2/15
9598/9598 [==============================] - 81s - loss: 2.1752 - acc: 0.3992 - val_loss: 2.0794 - val_acc: 0.4133
Epoch 3/15
9598/9598 [==============================] - 81s - loss: 2.0457 - acc: 0.4309 - val_loss: 1.9940 - val_acc: 0.4405
Epoch 4/15
9598/9598 [==============================] - 81s - loss: 1.9617 - acc: 0.4495 - val_loss: 1.9450 - val_acc: 0.4574
Epoch 5/15
9598/9598 [==============================] - 81s - loss: 1.9040 - acc: 0.4640 - val_loss: 1.9060 - val_acc: 0.4592
Epoch 6/15
9598/9598 [==============================] - 80s - loss: 1.8473 - acc: 0.4772 - val_loss: 1.8977 - val_acc: 0.4574
Epoch 7/15
9598/9598 [==============================] - 81s - loss: 1.8107 - acc: 0.4879 - val_loss: 1.8660 - val_acc: 0.4602
Epoch 8/15
9598/9598 [==============================] - 80s - lo

In [16]:
#predict data
out=predict_data()

/root/workspace/densenet161.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), name="conv1", strides=(2, 2), use_bias=False)`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv2_1_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv2_1_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:165: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'

In [17]:
p_path="data/test"
image_paths = sorted(list(paths.list_images(p_path)))
image_name=[]
for im_path in image_paths:
    image_name.append(im_path.split('/')[2])
predict_label = pd.read_csv("data/submit_example.csv")
for i in range(predict_label.values.shape[0]):
    find_name=predict_label.values[i][0]
    for j in range(len(image_name)):
        if find_name==image_name[j]:
            tmp_out=out[j]
            list_t=[]
            for m in range(tmp_out.shape[0]):
                if tmp_out[m]>0.2:
                    list_t.append(m+1)
            if len(list_t)==0:
                print(image_name[j])
                list_t.append(np.argmax(tmp_out))
            str_s=""
            for n in range(len(list_t)):
                str_s=str_s+str(list_t[n])
                if n!=len(list_t)-1:
                    str_s=str_s+";"
            predict_label.iloc[i,1]=str_s
predict_label.to_csv("data/predict.csv",index=False)

0203df2028d748328b1cb47cd2590fd6.png
0222be7a599f4b98b655628d3813df2e.jpg
046f83dfbdc7432496fc2c9ad7f0d370.png
06a7d584bd5044a3b5b66858f48e2e51.jpg
06d7908e9ba743d8b68dfb65d81eeb30.png
0925377d583741078368fb69a0593de5.jpg
09e179c8b9374a14bb92be752334f5fd.png
0a043bbe2e3d4c7ea3bdcf07b579b768.png
0a91b0cab4214e01bd8000db56220ffb.png
0b70d17f3cdd45258e603daca0edacd6.png
0bb00aa0027c42c8a3c221a1ff03cad2.jpg
0e4e45675df94ae8803eb92b99d595f9.jpg
0f856ca8f8d84c59b6392134bc2fbdd4.jpg
109b1cee81a54b17b420a8acd6846834.jpg
11c970d7d3064b5cbd3f9f4a1f67eee9.jpg
1242103878a94900ba60f16ef638bf8a.jpg
12699e22da3840cd9124155302953cf8.jpg
133e208f0c4e4a4aa9aa35a6bb42fe94.png
14901da26f3142a9b63f9ef861248c47.png
14b77b330f794393b9aa61c69cf27a88.jpg
14ca266d87464388a6222a3663b9ce92.png
14d2dd7de332424e92b5c0135c75473b.png
153d19a34a224f398482e1a016a39ce7.jpg
158f25a387d34108aa0b488b973e7520.jpg
15c70ab9439f433489a676024a8503c9.jpg
161180997dde43d1b07088175ffd6263.jpg
1701ac57679942a899d7ed70a99de240.jpg
1